In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

from zsldataset import ZSLDataset
from models import ContinuousMap, ContinuousMapResidual

def dist_matrix(batch1, batch2):
    delta = batch1.unsqueeze(1) - batch2.unsqueeze(0)
    
    dist_matrix = (delta * delta).mean(dim=-1)
    
    return dist_matrix


def mag(u):
    return torch.dot(u, u)


def dist(u, v):
    return torch.dot(u - v, u - v)

ModuleNotFoundError: No module named 'torch'

## Load Yahoo dataset

Should be converted into ZSL format beforehand.

In [2]:
trainset = ZSLDataset('Data/APY/train', use_predicates=False, use_irevnet=True)
testset = ZSLDataset('Data/APY/test', use_predicates=False, use_irevnet=True)

In [14]:
bsize = 128
n_epochs = 200
num_classes = trainset.classes.shape[0]

dim_semantic = trainset[0]['class_embedding'].shape[0]
dim_visual = trainset[0]['image_embedding'].shape[0]

all_class_embeddings = torch.tensor(np.array(trainset.class_embeddings)).cuda().float()
classes_enum = torch.tensor(np.array(range(num_classes), dtype=np.int64)).cuda()

In [15]:
v_to_s = ContinuousMap(dim_source=dim_visual, dim_dest=dim_semantic, width=512).cuda()
s_to_v = ContinuousMap(dim_source=dim_semantic, dim_dest=dim_visual, width=512).cuda()

# optimizer = torch.optim.Adam(list(v_to_s.parameters()) + list(s_to_v.parameters()),
#                                 lr = 1e-3,
#                                 betas=(0.9, 0.999),
#                                 weight_decay=3e-2)

optimizer = torch.optim.SGD(list(v_to_s.parameters()) + list(s_to_v.parameters()),
                                lr = 1e-3,
                                momentum=.9,
                                weight_decay=3e-2)

In [16]:
mse = torch.nn.MSELoss()
triplet_loss = torch.nn.TripletMarginLoss(margin=1, p=2)
positive_part = torch.nn.ReLU()

In [2]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                             batch_size=bsize, 
                                             shuffle=True, 
                                             num_workers=4, 
                                             pin_memory=True, 
                                             drop_last=True)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=bsize, 
                                         shuffle=True, 
                                         num_workers=4, 
                                         pin_memory=True, 
                                         drop_last=True)

NameError: name 'trainset' is not defined

In [19]:
for e in range(n_epochs):
    v_to_s = v_to_s.train()
    s_to_v = s_to_v.train()
    
    for i, sample in enumerate(trainloader):
        optimizer.zero_grad()
        
        batch_visual = sample['image_embedding'].cuda().float()
        batch_semantic = sample['class_embedding'].cuda().float()

        batch_classes = sample['class_id'].cuda()
        
        e_hat = v_to_s(s_to_v(all_class_embeddings))
        delta = (e_hat - all_class_embeddings)
        surjection_loss = (delta * delta).sum(dim=-1).mean()
        
        s_out = v_to_s(batch_visual)
        
        same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
        same_class = same_class.detach()

        d_matrix = dist_matrix(s_out, all_class_embeddings)

        closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
        furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)
        
        l2_loss = (s_out * s_out).sum(dim=-1).mean()

        loss = positive_part(furthest_positive - closest_negative + 1)
        loss = 5 * loss.mean() + 1 * surjection_loss + 1e-3 * l2_loss
        
        loss.backward()
        
#         c_hat = d_matrix.argmin(dim = -1)
#         print((c_hat == batch_classes).float().mean().item())
        
        optimizer.step()
        
        print(loss.item(), end=', ')
        
    if (e+1) % 50 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.7
    
    if (e+1) % 5 == 0:
        print('\n\n- Evaluation on epoch {}'.format(e))
        
        avg_accuracy = 0.
        avg_loss = 0.
        n = 0
        
        v_to_s = v_to_s.eval() 
        s_to_v = s_to_v.eval() 
        
        with torch.no_grad():
            for i, sample in enumerate(testloader):
                n += 1
                
                batch_visual = sample['image_embedding'].cuda().float()
                batch_semantic = sample['class_embedding'].cuda().float()

                batch_classes = sample['class_id'].cuda()

                s_out = v_to_s(batch_visual)

                same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
                same_class = same_class.detach()

                d_matrix = dist_matrix(s_out, all_class_embeddings)
                
                c_hat = d_matrix.argmin(dim = -1)

                closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
                furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)

                loss = positive_part(furthest_positive - closest_negative + 5)
                loss = 10 * furthest_positive.mean()

                avg_loss += loss.item()
                avg_accuracy += (c_hat == batch_classes).float().mean().item()

        avg_accuracy /= n
        avg_loss /= n

        print('Average acc.: {}, Average loss:{}\n\n'.format(avg_accuracy, avg_loss))


80.53130340576172, 61.464500427246094, 52.31481170654297, 46.25139236450195, 41.29546356201172, 37.05146026611328, 33.3905143737793, 30.08965301513672, 27.169281005859375, 24.604129791259766, 22.38240623474121, 20.48441505432129, 18.887699127197266, 17.53232192993164, 16.375404357910156, 15.378554344177246, 14.508137702941895, 13.738895416259766, 13.071162223815918, 12.442060470581055, 11.90821647644043, 11.398086547851562, 10.938286781311035, 10.485614776611328, 10.085139274597168, 9.705642700195312, 9.360555648803711, 9.035926818847656, 8.724611282348633, 8.438621520996094, 8.176355361938477, 7.932048797607422, 7.713849067687988, 7.483469486236572, 7.313209533691406, 7.130200386047363, 6.975046634674072, 6.820808410644531, 6.6933488845825195, 6.576348304748535, 6.4489216804504395, 6.339223861694336, 6.2363715171813965, 6.159274578094482, 6.078036308288574, 6.02764368057251, 5.952661037445068, 5.898092269897461, 5.852380752563477, 5.801645755767822, 5.761543273925781, 5.73233890533447

5.347219944000244, 5.3484787940979, 5.348219394683838, 5.3458333015441895, 5.349427700042725, 5.348209381103516, 5.3488569259643555, 5.346475124359131, 5.343312740325928, 5.351103782653809, 5.353764057159424, 5.346197128295898, 5.3471550941467285, 5.34688663482666, 5.34242057800293, 5.347638130187988, 5.344573974609375, 5.3454389572143555, 5.347228527069092, 5.348572731018066, 5.34894323348999, 5.3511223793029785, 5.344875812530518, 5.347209930419922, 5.347202777862549, 5.349666118621826, 5.348479747772217, 5.345147609710693, 5.347986221313477, 5.347705841064453, 5.3486857414245605, 5.3490729331970215, 5.34285306930542, 5.345241546630859, 5.350532054901123, 5.3507184982299805, 5.3452253341674805, 5.344748020172119, 5.346439361572266, 5.3499674797058105, 5.349027633666992, 5.349132537841797, 5.344058513641357, 5.343698024749756, 5.34756326675415, 5.34488582611084, 5.346581935882568, 5.349754333496094, 5.346730709075928, 5.3455915451049805, 5.340796947479248, 5.346189498901367, 5.3390579

5.32249641418457, 5.323193550109863, 5.323827743530273, 5.3271942138671875, 5.325189590454102, 5.327195644378662, 5.3201823234558105, 5.329107761383057, 5.326282024383545, 5.327115058898926, 5.323447227478027, 5.326258659362793, 5.3272271156311035, 5.324306964874268, 5.3297295570373535, 5.329155921936035, 5.323251247406006, 5.325973033905029, 5.326803684234619, 5.322943687438965, 5.322973728179932, 5.321162223815918, 5.325220584869385, 5.327645778656006, 5.33107328414917, 5.324530601501465, 5.32492733001709, 5.326945781707764, 5.323224067687988, 5.324290752410889, 5.323122978210449, 5.324008464813232, 5.3250813484191895, 5.326141834259033, 5.329038619995117, 5.3227081298828125, 5.32741117477417, 5.325680732727051, 5.326691150665283, 5.3302412033081055, 5.326984882354736, 5.324194431304932, 5.324359893798828, 5.322657585144043, 5.326552391052246, 5.3240814208984375, 5.322039604187012, 5.323476314544678, 5.329999923706055, 5.324888229370117, 5.322475910186768, 5.326184272766113, 5.324440

5.316614627838135, 5.318078994750977, 5.314319610595703, 5.318953037261963, 5.32306432723999, 5.313277721405029, 5.313700199127197, 5.319398880004883, 5.320396900177002, 5.316638469696045, 5.320499897003174, 5.318105697631836, 5.314160346984863, 5.319375038146973, 5.313791275024414, 5.312841892242432, 5.320905685424805, 5.316248893737793, 5.318492889404297, 5.314824104309082, 5.31736946105957, 5.323293685913086, 5.319465160369873, 5.318055152893066, 5.32064151763916, 5.3146796226501465, 5.318911552429199, 5.319226264953613, 5.313939094543457, 5.314634799957275, 5.313192844390869, 5.316274642944336, 5.316256999969482, 5.3210225105285645, 5.316710948944092, 5.321852207183838, 5.319353103637695, 5.317612171173096, 5.312037944793701, 5.313561916351318, 5.31587028503418, 5.317632675170898, 5.315019130706787, 5.319278717041016, 5.312745094299316, 5.314643859863281, 5.3143486976623535, 5.313208103179932, 5.314911365509033, 5.31143856048584, 5.317334175109863, 5.323182106018066, 5.315241813659

5.313802242279053, 5.3077545166015625, 5.324514389038086, 5.305271625518799, 5.3081464767456055, 5.321452617645264, 5.306585788726807, 5.308182239532471, 5.311519145965576, 5.319786548614502, 5.310515403747559, 5.306869983673096, 5.317288398742676, 5.313002586364746, 5.304397106170654, 5.31554651260376, 5.316044807434082, 5.315653324127197, 5.305945873260498, 5.309701919555664, 5.312220096588135, 5.316007137298584, 5.304656982421875, 5.308373928070068, 5.312562942504883, 5.312304496765137, 5.311214447021484, 5.313920021057129, 5.31095552444458, 5.318551063537598, 5.316866397857666, 5.325403213500977, 5.3134765625, 5.308079719543457, 5.313737392425537, 5.315323352813721, 5.3110198974609375, 5.312010765075684, 5.317008018493652, 5.319262504577637, 5.30819845199585, 5.310121059417725, 5.312639236450195, 5.308992862701416, 5.314164161682129, 5.316502571105957, 5.312943458557129, 5.315999507904053, 5.310680866241455, 5.310793876647949, 5.307735443115234, 5.305457592010498, 5.312688827514648

...

5.285554885864258, 5.285081386566162, 5.284095287322998, 5.285839557647705, 5.289685249328613, 5.285434246063232, 5.282455921173096, 5.281316757202148, 5.280035018920898, 5.28067684173584, 5.286718845367432, 5.289925575256348, 5.2819108963012695, 5.281646728515625, 5.2841925621032715, 5.286380767822266, 5.281574726104736, 5.283458232879639, 5.288026332855225, 5.2932634353637695, 5.301950931549072, 5.284459114074707, 5.292311191558838, 5.324997901916504, 5.2876739501953125, 5.291623115539551, 5.285612106323242, 5.285472393035889, 5.28406286239624, 5.2852559089660645, 5.286391258239746, 5.2838945388793945, 5.280895233154297, 5.288623332977295, 5.28674840927124, 5.283666610717773, 5.283073902130127, 5.286246299743652, 5.294461250305176, 5.28191614151001, 5.301656723022461, 5.301421642303467, 5.286514759063721, 5.283350944519043, 5.2808356285095215, 5.285606861114502, 5.281828880310059, 5.2799296379089355, 5.28539514541626, 5.290144443511963, 5.283718109130859, 5.286533355712891, 5.2890138

5.285595893859863, 5.281613349914551, 5.287056922912598, 5.283167839050293, 5.283119201660156, 5.2811431884765625, 5.28582239151001, 5.290535926818848, 5.27972936630249, 5.284883499145508, 5.281879901885986, 5.324654579162598, 5.286787033081055, 5.282172679901123, 5.2812275886535645, 5.284367084503174, 5.2859649658203125, 5.284701347351074, 5.279548645019531, 5.2840256690979, 5.288261413574219, 5.281773090362549, 5.283754825592041, 5.279829025268555, 5.280512809753418, 5.2819671630859375, 5.286904811859131, 5.280013561248779, 5.284544944763184, 5.293599605560303, 5.28077507019043, 5.28810977935791, 5.307648181915283, 5.280623435974121, 5.290070533752441, 5.285806179046631, 5.282602310180664, 5.280157089233398, 5.293459892272949, 5.287947654724121, 5.287317752838135, 5.282078742980957, 5.286509037017822, 5.2923455238342285, 5.28014612197876, 5.289963245391846, 5.289613246917725, 5.282794952392578, 5.285767555236816, 5.281796932220459, 5.282716751098633, 5.284247398376465, 5.289864063262

5.302675724029541, 5.304707050323486, 5.290632724761963, 5.280545711517334, 5.282145977020264, 5.283517360687256, 5.291756629943848, 5.291228294372559, 5.290421485900879, 5.286897659301758, 5.281375885009766, 5.288186073303223, 5.294173717498779, 5.279687881469727, 5.280736923217773, 5.282138824462891, 5.286285877227783, 5.294733047485352, 5.281612873077393, 5.281806945800781, 5.280159950256348, 5.282934188842773, 5.2928667068481445, 5.290183067321777, 5.285323143005371, 5.2898149490356445, 5.283051013946533, 5.281675815582275, 5.29157018661499, 5.284125328063965, 5.2871551513671875, 5.288821220397949, 5.280606269836426, 5.300189971923828, 5.289447784423828, 5.279890537261963, 5.286585330963135, 5.282743453979492, 5.284510612487793, 5.281052112579346, 5.289951801300049, 5.286721706390381, 5.280183792114258, 5.283960819244385, 5.282017707824707, 5.2876996994018555, 5.283308506011963, 5.284966945648193, 5.282046318054199, 5.281793117523193, 5.2805399894714355, 5.283120155334473, 5.284085

5.281505584716797, 5.296708106994629, 5.28584623336792, 5.280641078948975, 5.290338039398193, 5.292797088623047, 5.289380073547363, 5.2841362953186035, 5.28367805480957, 5.2917633056640625, 5.284324645996094, 5.287592887878418, 5.282325267791748, 5.294152736663818, 5.287627696990967, 5.302420616149902, 5.281466960906982, 5.291613578796387, 5.285043716430664, 5.284110069274902, 5.284000396728516, 5.281407356262207, 5.2880072593688965, 5.281793117523193, 5.280178070068359, 5.284974575042725, 5.279505252838135, 5.280714988708496, 5.2857489585876465, 5.284689426422119, 5.2803826332092285, 5.2843828201293945, 5.294157028198242, 5.281071662902832, 5.282942771911621, 5.283910274505615, 5.279490947723389, 5.280943870544434, 5.301426410675049, 5.285144329071045, 5.279240608215332, 5.282416820526123, 5.282238006591797, 5.281458854675293, 5.295642852783203, 5.283836841583252, 5.288550853729248, 5.299310207366943, 5.300135612487793, 5.281826019287109, 5.281708717346191, 5.294300556182861, 5.293690

5.278763771057129, 5.284741401672363, 5.290650844573975, 5.2815093994140625, 5.289682865142822, 5.28262996673584, 5.279788494110107, 5.32492733001709, 5.282701015472412, 5.295329570770264, 5.281108379364014, 5.283209800720215, 5.283419132232666, 5.28533935546875, 5.285211563110352, 5.280477523803711, 5.289200305938721, 5.2825541496276855, 5.285958290100098, 5.28214693069458, 5.279839038848877, 5.281244277954102, 5.291326522827148, 5.280198574066162, 5.284478664398193, 5.279074668884277, 5.279417991638184, 5.2890944480896, 5.30031681060791, 5.279354572296143, 5.286403179168701, 5.296128273010254, 5.285741329193115, 5.27996301651001, 5.278993129730225, 5.27993106842041, 5.28931188583374, 5.28495979309082, 5.291201591491699, 5.28557825088501, 5.302340507507324, 5.285962104797363, 5.286319732666016, 5.287452220916748, 5.282742977142334, 5.279848575592041, 5.284427165985107, 5.28560733795166, 5.288464546203613, 5.28622579574585, 5.28990364074707, 5.280708312988281, 5.290557384490967, 5.2861

# Best results

with words only: 34.6%

with words+attributes:

# With attributes

In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

from zsldataset import ZSLDataset
from models import ContinuousMap, ContinuousMapResidual

def dist_matrix(batch1, batch2):
    delta = batch1.unsqueeze(1) - batch2.unsqueeze(0)
    
    dist_matrix = (delta * delta).mean(dim=-1)
    
    return dist_matrix


def mag(u):
    return torch.dot(u, u)


def dist(u, v):
    return torch.dot(u - v, u - v)

In [2]:
from models import EncoderAttributes, DecoderAttributes

In [3]:
trainset = ZSLDataset('Data/APY/train', use_predicates=True, use_irevnet=True)
testset = ZSLDataset('Data/APY/test', use_predicates=True, use_irevnet=True)

In [4]:
bsize = 128
n_epochs = 200
num_classes = trainset.classes.shape[0]

dim_semantic = trainset[0]['class_embedding'].shape[0]
dim_visual = trainset[0]['image_embedding'].shape[0]
dim_attributes = trainset[0]['class_predicates'].shape[0]

all_class_embeddings = torch.tensor(np.array(trainset.class_embeddings)).cuda().float()
all_class_predicates = torch.tensor(np.array(trainset.class_predicates)).cuda().float()
classes_enum = torch.tensor(np.array(range(num_classes), dtype=np.int64)).cuda()

In [17]:
v_to_s = DecoderAttributes(dim_source=dim_visual, dim_target1=dim_attributes, dim_target2=dim_semantic, width=512).cuda()
s_to_v = EncoderAttributes(dim_source1=dim_semantic, dim_source2=dim_attributes, dim_target=dim_visual, width=512).cuda()

# optimizer = torch.optim.Adam(list(v_to_s.parameters()) + list(s_to_v.parameters()),
#                                 lr = 1e-3,
#                                 betas=(0.9, 0.999),
#                                 weight_decay=3e-2)

optimizer = torch.optim.SGD(list(v_to_s.parameters()) + list(s_to_v.parameters()),
                                lr = 1e-3,
                                momentum=.9,
                                weight_decay=3e-2,
                                nesterov=True)

In [18]:
mse = torch.nn.MSELoss()
triplet_loss = torch.nn.TripletMarginLoss(margin=1, p=2)
positive_part = torch.nn.ReLU()

In [19]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                             batch_size=bsize, 
                                             shuffle=True, 
                                             num_workers=4, 
                                             pin_memory=True, 
                                             drop_last=True)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=bsize, 
                                         shuffle=True, 
                                         num_workers=4, 
                                         pin_memory=True, 
                                         drop_last=True)

In [16]:
gamma = 0.7

alpha1 = 5 # triplet
alpha2 = 3 # surjection
alpha3 = 1e-3 # l2 loss


In [ ]:
for e in range(n_epochs):
    v_to_s = v_to_s.train()
    s_to_v = s_to_v.train()
    
    for i, sample in enumerate(trainloader):
        optimizer.zero_grad()
        
        batch_visual = sample['image_embedding'].cuda().float()
        batch_semantic = sample['class_embedding'].cuda().float()

        batch_classes = sample['class_id'].cuda()
         
        e_hat = v_to_s(s_to_v(all_class_embeddings, all_class_predicates))
        delta = (e_hat[1] - all_class_embeddings)
        surjection_loss = (delta * delta).sum(dim=-1).mean()
        delta = (e_hat[0] - all_class_predicates)
        surjection_loss = (1-gamma) * surjection_loss + gamma * (delta * delta).sum(dim=-1).mean()
        
        s_out = v_to_s(batch_visual)
        s_attr, s_word = s_out
        
        same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
        same_class = same_class.detach()

        d_matrix = (1 - gamma) * dist_matrix(s_word, all_class_embeddings) + \
                    gamma * dist_matrix(s_attr, all_class_predicates)
        
        closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
        furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)
        
        l2_loss = (1-gamma) * (s_word * s_word).sum(dim=-1).mean() + \
                    gamma * (s_attr * s_attr).sum(dim=-1).mean()

        loss = positive_part(furthest_positive - closest_negative + 1)
        loss = alpha1 * loss.mean() + alpha2 * surjection_loss + alpha3 * l2_loss
        
        loss.backward()
        
#         c_hat = d_matrix.argmin(dim = -1)
#         print((c_hat == batch_classes).float().mean().item())
        
        optimizer.step()
        
        print(loss.item(), end=', ')
        
    if (e+1) % 50 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.7
    
    if (e+1) % 5 == 0:
        print('\n\n- Evaluation on epoch {}'.format(e))
        
        avg_accuracy = 0.
        avg_loss = 0.
        n = 0
        
        v_to_s = v_to_s.eval() 
        s_to_v = s_to_v.eval() 
        
        with torch.no_grad():
            for i, sample in enumerate(testloader):
                n += 1
                
                batch_visual = sample['image_embedding'].cuda().float()
                batch_semantic = sample['class_embedding'].cuda().float()

                batch_classes = sample['class_id'].cuda()

                s_out = v_to_s(batch_visual)
                s_attr, s_word = s_out

                same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
                same_class = same_class.detach()

                d_matrix = (1 - gamma) * dist_matrix(s_word, all_class_embeddings) + \
                                gamma * dist_matrix(s_attr, all_class_predicates)
                
                c_hat = d_matrix.argmin(dim = -1)

                closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
                furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)

                loss = positive_part(furthest_positive - closest_negative + 5)
                loss = alpha1 * furthest_positive.mean()

                avg_loss += loss.item()
                avg_accuracy += (c_hat == batch_classes).float().mean().item()

        avg_accuracy /= n
        avg_loss /= n

        print('Average acc.: {}, Average loss:{}\n\n'.format(avg_accuracy, avg_loss))




87.82241821289062, 62.26003646850586, 49.6287727355957, 42.43606185913086, 37.141700744628906, 32.91790008544922, 29.410888671875, 26.461647033691406, 23.989721298217773, 21.89542579650879, 20.105852127075195, 18.57509994506836, 17.259981155395508, 16.112829208374023, 15.117913246154785, 14.238020896911621, 13.457895278930664, 12.762311935424805, 12.132343292236328, 11.56953239440918, 11.053337097167969, 10.585325241088867, 10.15835189819336, 9.757954597473145, 9.394582748413086, 9.060996055603027, 8.749269485473633, 8.465121269226074, 8.20019817352295, 7.951857566833496, 7.716302394866943, 7.5130438804626465, 7.3103413581848145, 7.127112865447998, 6.960219860076904, 6.7988691329956055, 6.660757064819336, 6.527007579803467, 6.400520324707031, 6.28937292098999, 6.182424068450928, 6.0861334800720215, 5.9967474937438965, 5.915666103363037, 5.840953350067139, 5.775514125823975, 5.715530872344971, 5.65854549407959, 5.60286283493042, 5.56473970413208, 5.519048690795898, 5.477886199951172, 5.

5.160826683044434, 5.163203716278076, 5.163897514343262, 5.164052963256836, 5.1639180183410645, 5.16202449798584, 5.161894798278809, 5.164618492126465, 5.1611714363098145, 5.163522720336914, 5.161015033721924, 5.1636505126953125, 5.15999174118042, 5.162708282470703, 5.163854122161865, 5.16277551651001, 5.163585186004639, 5.163949966430664, 5.159905910491943, 5.159695148468018, 5.1622185707092285, 5.162734031677246, 5.162425994873047, 5.161837577819824, 5.162049293518066, 5.16146993637085, 5.162776470184326, 5.163761138916016, 5.162519931793213, 5.1620635986328125, 5.159128665924072, 5.163061141967773, 5.16117000579834, 5.1613264083862305, 5.161298751831055, 5.160159111022949, 5.162942409515381, 5.159888744354248, 5.158681392669678, 5.160549640655518, 5.161250591278076, 5.160757541656494, 5.15759801864624, 5.162252426147461, 5.159707546234131, 5.16133975982666, 5.158064842224121, 5.161064624786377, 5.160437107086182, 5.158450603485107, 5.159770488739014, 5.159142971038818, 5.15708875656

5.147297382354736, 5.144930362701416, 5.146640777587891, 5.14943790435791, 5.145772933959961, 5.147166728973389, 5.145120143890381, 5.147459983825684, 5.146928310394287, 5.145688056945801, 5.14328670501709, 5.143631458282471, 5.148410797119141, 5.145203590393066, 5.144901752471924, 5.148796081542969, 5.147865295410156, 5.144099235534668, 5.145944595336914, 5.146703720092773, 5.146109104156494, 5.147430419921875, 5.145608425140381, 5.146472454071045, 5.146896839141846, 5.143812656402588, 5.146900653839111, 5.145994186401367, 5.147776126861572, 5.1458516120910645, 5.1459479331970215, 5.146414756774902, 5.145889759063721, 5.147974014282227, 5.145176410675049, 5.146575450897217, 5.1459245681762695, 5.146173000335693, 5.1435980796813965, 5.146746635437012, 5.143766403198242, 5.143519401550293, 5.14376974105835, 5.146334171295166, 5.14512825012207, 5.147592067718506, 5.146142959594727, 5.1435322761535645, 5.1465654373168945, 5.141574382781982, 5.144099235534668, 5.145453929901123, 5.14448451

5.140572547912598, 5.140594005584717, 5.137659549713135, 5.1373610496521, 5.140755653381348, 5.138110637664795, 5.136977672576904, 5.139089584350586, 5.140374183654785, 5.135600566864014, 5.136563301086426, 5.138667583465576, 5.138136863708496, 5.139786243438721, 5.139062881469727, 5.138550758361816, 5.137160301208496, 5.1388115882873535, 5.138119220733643, 5.140671253204346, 5.138471603393555, 5.138824462890625, 5.140256881713867, 5.1386919021606445, 5.135818004608154, 5.140230178833008, 5.140091419219971, 5.141474723815918, 5.138688087463379, 5.140564441680908, 5.138636112213135, 5.141055107116699, 5.139101028442383, 5.13674259185791, 5.137861728668213, 5.140494346618652, 5.14040470123291, 5.139411449432373, 5.138782024383545, 5.136845588684082, 5.138784885406494, 5.138826370239258, 5.139638900756836, 5.139010429382324, 5.139456272125244, 5.1404337882995605, 5.139618396759033, 5.140048027038574, 5.138370037078857, 5.137675762176514, 5.140243053436279, 5.135920524597168, 5.13930797576

5.136264324188232, 5.132960319519043, 5.1360931396484375, 5.135782718658447, 5.135934829711914, 5.136119365692139, 5.131826877593994, 5.136024475097656, 5.131152153015137, 5.135335922241211, 5.137600421905518, 5.133971214294434, 5.135434150695801, 5.136708736419678, 5.136996269226074, 5.133505821228027, 5.133094310760498, 5.133705139160156, 5.13312292098999, 5.134262561798096, 5.137202739715576, 5.13283109664917, 5.138998508453369, 5.128868103027344, 5.135915756225586, 5.134373664855957, 5.135581970214844, 5.137803554534912, 5.135426998138428, 5.135174751281738, 5.133374214172363, 5.131632328033447, 5.1360859870910645, 5.13181209564209, 5.137819766998291, 5.1321234703063965, 5.137752532958984, 5.136106014251709, 5.137861251831055, 5.133669376373291, 5.131638050079346, 5.135254383087158, 5.1317315101623535, 5.136373043060303, 5.136760711669922, 5.131821632385254, 5.135326385498047, 5.136606693267822, 5.131296157836914, 5.129568576812744, 5.135730266571045, 5.133998870849609, 5.133708000

...

5.1185197830200195, 5.1109185218811035, 5.109501838684082, 5.11073637008667, 5.1245880126953125, 5.113673210144043, 5.112204551696777, 5.109859943389893, 5.132562160491943, 5.110035419464111, 5.115731239318848, 5.113295555114746, 5.114342212677002, 5.1361799240112305, 5.113959789276123, 5.115902900695801, 5.1112518310546875, 5.111691951751709, 5.111192226409912, 5.116488933563232, 5.117562294006348, 5.109880447387695, 5.111952304840088, 5.115128993988037, 5.137609004974365, 5.11298942565918, 5.116340160369873, 5.11714506149292, 5.116842269897461, 5.109994888305664, 5.114950180053711, 5.126014232635498, 5.115196228027344, 5.11807107925415, 5.111831188201904, 5.133335113525391, 5.111621856689453, 5.110952377319336, 5.115097522735596, 5.110467910766602, 5.110520362854004, 5.115089416503906, 5.110134124755859, 5.1209187507629395, 5.118320465087891, 5.110270977020264, 5.110762119293213, 5.110754489898682, 5.111494064331055, 5.1160383224487305, 5.114300727844238, 5.113485813140869, 5.1193618

5.115325450897217, 5.113465785980225, 5.110619068145752, 5.109237194061279, 5.112066268920898, 5.114709377288818, 5.110367774963379, 5.112768650054932, 5.1099348068237305, 5.112015247344971, 5.109933853149414, 5.118496894836426, 5.11916446685791, 5.111605167388916, 5.108997344970703, 5.1154327392578125, 5.109705448150635, 5.113119602203369, 5.114065170288086, 5.117703914642334, 5.136604309082031, 5.118101119995117, 5.113382339477539, 5.111664295196533, 5.116878986358643, 5.1111345291137695, 5.119444847106934, 5.111080169677734, 5.109630584716797, 5.117859840393066, 5.108564853668213, 5.113853931427002, 5.112130165100098, 5.115298748016357, 5.111953258514404, 5.108327865600586, 5.116366863250732, 5.117858409881592, 5.118828773498535, 5.11262321472168, 5.10999870300293, 5.111281871795654, 5.113504886627197, 5.109347343444824, 5.1188859939575195, 5.137939453125, 5.11039924621582, 5.111690044403076, 5.109461784362793, 5.109799385070801, 5.115224361419678, 5.114997386932373, 5.1110196113586

5.112549781799316, 5.113306045532227, 5.109035491943359, 5.108114719390869, 5.122283458709717, 5.112797737121582, 5.117928504943848, 5.1088972091674805, 5.124367713928223, 5.109280109405518, 5.110357284545898, 5.109179973602295, 5.11892557144165, 5.112990856170654, 5.120372772216797, 5.1158833503723145, 5.116921424865723, 5.121464252471924, 5.117554187774658, 5.111814498901367, 5.115365505218506, 5.113228797912598, 5.109694957733154, 5.109945297241211, 5.1111650466918945, 5.111001014709473, 5.109925270080566, 5.1092352867126465, 5.113507270812988, 5.11277961730957, 5.109250545501709, 5.109453201293945, 5.112998962402344, 5.11001443862915, 5.109383583068848, 5.1105451583862305, 5.112392902374268, 5.112165927886963, 5.11397123336792, 5.1260666847229, 5.110605239868164, 5.1092329025268555, 5.110319137573242, 5.1195831298828125, 5.1119537353515625, 5.111039638519287, 5.113923072814941, 5.10984468460083, 5.11030387878418, 5.1195149421691895, 5.108800888061523, 5.1127729415893555, 5.11575126

5.1157379150390625, 5.115384578704834, 5.108548641204834, 5.109619140625, 5.110850811004639, 5.113431930541992, 5.109865665435791, 5.113527297973633, 5.110812664031982, 5.108651638031006, 5.112614154815674, 5.1259446144104, 5.112094879150391, 5.1079301834106445, 5.108973979949951, 5.113785266876221, 5.1103715896606445, 5.118494033813477, 5.123514652252197, 5.11309289932251, 5.119716167449951, 5.1195173263549805, 5.120918273925781, 5.110698699951172, 5.108929634094238, 5.113207817077637, 5.110881805419922, 5.108707904815674, 5.108829021453857, 5.107462406158447, 5.1103692054748535, 5.114180088043213, 5.118061542510986, 5.1173200607299805, 5.113101482391357, 5.11361026763916, 5.109360694885254, 5.108765602111816, 5.122067451477051, 5.123758792877197, 5.114638805389404, 5.108938217163086, 5.120028972625732, 5.110004901885986, 5.117354869842529, 5.107656955718994, 5.110071659088135, 5.110238075256348, 5.113676071166992, 5.107569694519043, 5.1111297607421875, 5.108921527862549, 5.1207389831

5.1104021072387695, 5.113773345947266, 5.111225605010986, 5.10900354385376, 5.109053134918213, 5.124011993408203, 5.114459037780762, 5.116929054260254, 5.116901397705078, 5.121592998504639, 5.110956192016602, 5.111461639404297, 5.108496189117432, 5.11385440826416, 5.115571975708008, 5.116950988769531, 5.116982936859131, 5.1118340492248535, 5.1197052001953125, 5.107906818389893, 5.118460178375244, 5.11156702041626, 5.110401630401611, 5.1129655838012695, 5.110232830047607, 5.121921539306641, 5.109245777130127, 5.115682601928711, 5.107963562011719, 5.118650436401367, 5.111399173736572, 5.110018730163574, 5.109408378601074, 5.116532802581787, 5.112954616546631, 5.110746383666992, 5.108073711395264, 5.109073638916016, 5.11047887802124, 5.10864782333374, 5.110869407653809, 5.107965469360352, 5.108790874481201, 5.107970714569092, 5.1099162101745605, 5.110569953918457, 5.108999252319336, 5.108760833740234, 5.123857021331787, 5.109496116638184, 5.109617710113525, 5.111488342285156, 5.1099801063